In [2]:
import ee
import geemap
import geemap.colormaps
import ipywidgets as w
from IPython.display import display

In [3]:
import geemap.toolbar

cmp = geemap.colormaps.palettes

In [9]:
Map = geemap.Map(center=(-5,-64), zoom=5, #basemap = 'TERRAIN',
                 draw_ctrl=False, attribution_ctrl=False, data_ctrl=False)
Map.layout.height = 'auto'

# Select only toolbar controls "inspector[0]" and "change_basemap[8]"
Map.toolbar.children = (Map.toolbar.children[0], Map.toolbar.children[8], Map.toolbar.children[-2])
Map.toolbar.layout.grid_template_rows='32px'

def show_map(map2show):
    try:
        Map.remove_colorbar()
    except:
        pass
    
    img = map2show[0]
    vmin = map2show[1]
    vmax = map2show[2]
    mtype = map2show[3]
    
    
    if vmin>0:
        img = img.updateMask(img.gte(vmin)).rename('value')
    else:
        img = img.updateMask(img.gt(vmin)).rename('value')
    img = img.updateMask(img.lte(vmax)).rename('value')
    
    if mtype == 'binary':
        parvis = {'min': vmin, 'max': vmin, 'palette': cmp.viridis_r}
        img = img.where(img, 1)
        Map.addLayer(img, parvis, 'Inundation map')
    else:
        parvis = {'min': vmin, 'max': vmax, 'palette': cmp.viridis_r}
        Map.addLayer(img, parvis, 'Inundation map')
        Map.add_colorbar(parvis, label = None, discrete=False, orientation='vertical', layer_name = 'Inundation map')

elements_width = '97%'
# Description
title = w.Image(value=open('../logos/logo_app.png', 'rb').read(),  layout=w.Layout(width='75%', height='auto'))
subtitle = w.HTML(value='<{size}>Displaying the inundation extent of the largest river basin on Earth <br/> WebGIS platform | Version Beta</{size}>'.format(size='h3'))
description = w.HTML(value= 
'<h4 align="justify">The Amazon-Inundation WebGIS App allows the visualization of the inundation extent across the Amazon River basin '
         'based on 29 datasets that were developed in the last decades with different spatial and temporal resolutions. </h4>'
                     '<p style="font-size:12px;">Developed by Rafael Fontana and Ayan Fleischmann <br/>'
                     'Institute of Hydraulic Research, Federal University of Rio Grande do Sul</p>')
description.layout.width = elements_width

about = w.HTML('<{size} align="justify">The Amazon River basin is the largest on Earth, and harbors unique freshwater ecosystems on which thousands of people rely for their livelihoods, '
                     'contrasting ecosystems are adapted to its seasonal inundation dynamics, and major biogeochemical processes occur. <br/>'
                     
'The Amazon-Inundation WebGIS App provides an easy way to visualize the inundation extent across the basin based on various datasets that were developed in the last decades. <br/>'
                     
'The products have different data sources, from passive microwave and synthetic aperture radar to optical imagery and hydrological modeling, with spatial resolution varying from 12.5 m to 25 km. <br/>'

'Some products are dynamic, which are shown here as long-term flood frequency maps, i.e., the higher the pixel value, the longer it is flooded throughout the hydrological cycle. <br/>'

'Two 1 km consensus maps are also provided, which show, for each 1 km pixel, how many datasets agree that it is flooded considering the long-term maximum or minimum inundation maps. <br/>'

'References and data availability are provided for each dataset. For more details, and to cite this tool, please refer to Fleischmann et al. (2021). <br/></{size}>'
                     .format(size='p'))
    
photo = w.Image(value=open('../logos/Photos.jpg', 'rb').read(),
                    width='97%',
                    height='auto')

logo_layout = w.Layout(width='auto', height='100px')

logo_hge = w.Image(value=open('../logos/logo_hge.png', 'rb').read(), layout=logo_layout)
logo_iph = w.Image(value=open('../logos/logo_iph.png', 'rb').read(), layout=logo_layout)
logo_ufrgs = w.Image(value=open('../logos/logo_ufrgs.png', 'rb').read(), layout=logo_layout)
logo_ird = w.Image(value=open('../logos/logo_ird.png', 'rb').read(), layout=logo_layout)

logos = w.HBox([logo_hge, logo_iph, logo_ufrgs, logo_ird])
#logos = w.HTML(value='<{size}><br/><br/><br/>Here will be the logos of the organizations that contributed with this development.<br/><br/><br/><br/></{size}>'.format(size='p'))

# Categories
catList = ['Agreement among datasets at 1 km', 'Regional/local scale', 'Basin scale']
catW = w.widgets.RadioButtons(options=catList, description='Category: ', layout=w.Layout(width='400px'))

# Maps
mapList = {'Agreement among datasets at 1 km':
            {'Maximum extent agreement': [ee.Image('users/rbfontana/inundation_amazon/AgreementMax'), 1, 14, 'frequency',
'Products agreement on maximum inundation (number of products).'
             ],
             'Minimum extent agreement': [ee.Image('users/rbfontana/inundation_amazon/AgreementMin'), 1, 10, 'frequency',
'Products agreement on minimum inundation (number of products).'
             ]
             },
           
           'Regional/local scale':
            {'Uatumã (Resende et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/Resende'), 0, 999, 'binary',
'Binary, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 12.5 m / Temporal resolution: irregular / Period: 2006-2011. <br/>'
'Resende et al., 2019. Massive tree mortality from flood pulse disturbances in Amazonian floodplain forests: The collateral effects of hydropower production.'
'Sci. Total Environ. 659, 587–598. <a href="https://doi.org/10.1016/j.scitotenv.2018.12.208" target="_blank"> https://doi.org/10.1016/j.scitotenv.2018.12.208</a>'
             ],
'Curuai (Arnesen et al., 2103)': [ee.Image('users/rbfontana/inundation_amazon/Arnesen_FloodFrequency'), 0, 1, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR), spatial resolution: 90 m, temporal resolution: irregular, period: 2006-2010. <br/>'
'Arnesen et al., 2013. Monitoring flood extent in the lower Amazon River floodplain using ALOS/PALSAR ScanSAR images.' 
'Remote Sens. Environ. 130, 51–61. https://doi.org/10.1016/j.rse.2012.10.035'
             ],
             'Mamirauá (Ferreira-Ferreira et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/Mamiraua_FloodDur'), 1, 7, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 12.5 m / Temporal resolution: irregular / period: 2006-2011. <br/>'
'On map, inundation days per year: <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 1 &emsp; <40 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 2 &emsp; 40-105 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 3 &emsp; 105-125 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 4 &emsp; 125-175 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 5 &emsp; 175-295 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 6 &emsp; 295-365 <br/>'
'&emsp; &emsp; &emsp; &emsp; &emsp; 7 &emsp; 365 <br/>'
'Ferreira-Ferreira et al., 2015. Combining ALOS/PALSAR derived vegetation structure and inundation patterns to characterize major vegetation types in the Mamirauá Sustainable Development Reserve, Central Amazon floodplain, Brazil. ' 
'Wetl. Ecol. Manag. 23, 41–59. https://doi.org/10.1007/s11273-014-9359-1'
             ],
             'Janauacá (Pinel et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/PinelFloodFrequency'), 0, 1, 'frequency',
'Flood frequency, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 30 m / Temporal resolution: irregular / Period: 2006-2011. <br/>'
'Pinel et al., 2019. Flooding dynamics within an Amazonian floodplain: water circulation patterns and inundation duration. '
'Water Resour. Res. 2019WR026081. https://doi.org/10.1029/2019WR026081'
             ],
             'Llanos de Moxos (Ovando et al. 2016)': [ee.Image('users/rbfontana/inundation_amazon/ovandoMODIS'), 0, 1, 'frequency',
'Flood frequency, MODIS (optical imagery). <br/>'
'Spatial resolution: 500 m / Temporal resolution: 8 days / Period: 2001-2014. <br/>'
'Ovando et al., 2016. Extreme flood events in the Bolivian Amazon wetlands. '
'J. Hydrol. Reg. Stud. 5, 293–308. https://doi.org/10.1016/j.ejrh.2015.11.004'
            ],
            },
           
           'Basin scale':
            {'ESA-CCI (Bontemps et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/esa20101500'), 150, 999, 'binary',
'Binary, multisatellite (optical and SAR). <br/>'
'Spatial resolution: 300 m / Temporal resolution: annual / Period: 1992-2015. <br/>'
'Bontemps et al., 2013. Consistent global land cover maps for climate modelling communities: current achievements of the ESA’s land cover CCI. '
'Proceedings of the ESA Living Planet Symposium. Edinburgh.'

             ],
             'GLWD (Lehner & Döll, 2004)': [ee.Image('users/rbfontana/inundation_amazon/glwd_Amaz500'), 0, 999, 'binary',
'Binary, fusion of multiple datasets. <br/>'
'Spatial resolution: 1 km / Temporal resolution: static / Period: 1992-2004. <br/>'
'Lehner & Döll, 2004. Development and validation of a global database of lakes, reservoirs and wetlands. '
'J. Hydrol. 296, 1–22. https://doi.org/10.1016/j.jhydrol.2004.03.028'
             ],
             'CIFOR (Gumbricht et al., 2017) ': [ee.Image('users/rbfontana/inundation_amazon/cifor_amaz500'), 0, 999, 'binary',
'Binary, fusion of multiple datasets. <br/>'
'Spatial resolution: 232 m / Temporal resolution: static / Period: 1950-2000. <br/>'
'Gumbricht et al., 2017. An expert system model for mapping tropical wetlands and peatlands reveals South America as the largest contributor. '
'Glob. Chang. Biol. 23, 3581–3599. https://doi.org/10.1111/gcb.13689'
             ],
             'MGB (Siqueira et al., 2018)': [ee.Image('users/rbfontana/inundation_amazon/MGB_AmazonBasin'), 0, 1, 'frequency',
'Flood frequency, hydrological model. <br/>'
'Spatial resolution: 500 m / Temporal resolution: monthly / Period: 1980-2014. <br/>'
'Siqueira et al., 2018. Toward continental hydrologic–hydrodynamic modeling in South America. '
'Hydrol. Earth Syst. Sci. 22, 4815–4842. https://doi.org/10.5194/hess-22-4815-2018'
             ],
             'GIEMS-D15 (Fluet-Chouinard et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/giems_amaz21500'), 0, 999, 'binary',
'Binary, fusion of multiple datasets. <br/>'
'Spatial resolution: 232 m / Temporal resolution: static / Period: 1993-2004. <br/>'
'Fluet-Chouinard et al., 2015. Development of a global inundation map at high spatial resolution from topographic downscaling of coarse-scale remote sensing data. '
'Remote Sens. Environ. 158, 348–361. https://doi.org/10.1016/j.rse.2014.10.015'
             ],
             'G3WBM (Yamazaki et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/g3wbm_amaz500'), 0, 999, 'binary',
'Binary, Landsat optical imagery. <br/>'
'Spatial resolution: 90 m / Temporal resolution: static / Period: 1990-2010. <br/>'
'Yamazaki et al., 2015. Development of a global ~90m water body map using multi-temporal Landsat images. '
'Remote Sens. Environ. 171, 337-351. https://doi.org/10.1016/j.rse.2015.10.014'
             ],
            'Chapman (Chapman et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/chapman_amaz_compres500'), 0, 4, 'binary',
'Binary, ALOS-PALSAR (L-band SAR). <br/>'
'Spatial resolution: 90 m / Temporal resolution: static / Period: 2006-2011. <br/>'
'Chapman et al., 2015. Mapping Regional Inundation with Spaceborne L-Band SAR. '
'Remote Sens. 7(5), 5440-5470. https://doi.org/10.3390/rs70505440'
            ],
            'Hess (Hess et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/hess_wetl_mask_compressed500'), 0, 999, 'binary',
'Binary, JERS-1 (L-band SAR). <br/>'
'Spatial resolution: 90 m / Temporal resolution: static / Period: 1995-1996. <br/>'
'Hess et al., 2015. Wetlands of the Lowland Amazon Basin: Extent, Vegetative Cover, and Dual-season Inundated Area as Mapped with JERS-1 Synthetic Aperture Radar. '
'Wetlands 35, 745–756. https://doi.org/10.1007/s13157-015-0666-y'
            ],
            }
        }

btnStyle = w.ButtonStyle(button_color='lightgreen')
btnLayout = w.Layout(width=elements_width)

btnMap = w.Button(description='Select map to display: ', layout=btnLayout, style=btnStyle)

mapDwn = w.widgets.Dropdown(options=mapList[catW.value], description='Product: ', layout=w.Layout(height='40px', width = '75%'))
def update_mapDwn(cat):
    mapDwn.options = mapList[cat.new]
catW.observe(update_mapDwn, 'value')
    
input_ops = w.interactive(show_map, map2show=mapDwn)

map_description = w.HTML(mapDwn.value[-1])
def update_map_description(product):
    map_description.value = product.new[-1]
mapDwn.observe(update_map_description, 'value')

btnDescription = w.Button(description='Product description: ', layout=btnLayout, style=btnStyle)
box_description = w.VBox([btnDescription, map_description])
#box_description = w.Accordion([ map_description])

btnAbout = w.Button(description='About the platform: ', layout=btnLayout, style=btnStyle)
about.layout.width = elements_width
box_about = w.VBox([w.HTML('<br/>'), btnAbout, about])

btnContact = w.Button(description='Contact: ', layout=btnLayout, style=btnStyle)
box_contact = w.VBox([w.HTML('<br/>'), btnContact, w.HTML('rbarbedofontana@gmail.com <br/> ayan.fleischmann@gmail.com')])

btnLogos = w.Button(description='Support: ', layout=btnLayout, style=btnStyle)
box_logos = w.VBox([w.HTML('<br/>'), btnLogos, logos])

# Final front
carousel_layout = w.Layout(#overflow='scroll hidden',
                    #border='3px solid black',
                    width='36%',
                    height='100%',
                    flex_flow='column',
                    display='flex-shrink')

carousel = w.Box(children=[title, subtitle, photo, description, btnMap, catW, input_ops, box_description, box_about, box_contact, box_logos], layout=carousel_layout)
carousel.layout.display = 'block' #Blocks the display visualization so items wont be hidden
# tab = w.Tab(children = [carousel, box_about], layout=carousel_layout)
# tab.layout.width = '625px'

box_layout = w.Layout(
    height='94vh',  # viewport height
    width='100%',
    grid_gap='5px',
    overflow='auto',
    align_items='stretch',
    margin='0px',
    border='0px',
    padding='0px',
)
box = w.HBox([carousel, Map], layout=box_layout)

display(box)
#Map